# 多自由度系の固有値解析

吉田勝俊（宇都宮大学）

## 参考情報
- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [【Python】行列指数関数・行列対数関数 - Qiita](https://qiita.com/Mrrmm252/items/a50a9b352e5064e40cc1)

In [ ]:
%matplotlib inline

import numpy as np                 #数値計算ライブラリ
from scipy.integrate import odeint #常微分方程式ライブラリ
import scipy.linalg as la          #線形代数ライブラリ
import matplotlib.pyplot as plt    #描画ライブラリ
plt_config = {
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'lines.linewidth': 1.2,
    'lines.markersize': 3,
    'lines.markeredgewidth': 0.7,
    'lines.markerfacecolor': 'white',
    'lines.markeredgecolor': 'black',
}
plt.rcParams.update(plt_config)

## ◯行列指数関数

### ■指数関数

In [ ]:
np.exp(2.5) #普通の指数関数

### ■行列指数関数

In [ ]:
A = np.array([
    [0, 1],
    [-3, -2]
])
display(A)

#### （誤）Numpy の `exp(行列)` $\neq$ 行列指数関数

In [ ]:
np.exp(A) #これは単なる各成分の指数関数値

In [ ]:
for i in range(2):
    for j in range(2):
        print(np.exp(A[i,j])) 

#### <font color="red">（正）Scipy の `expm(行列)` $=$ 行列指数関数</font>．

In [ ]:
la.expm(A)

## ◯解の表示（多次元）

In [ ]:
def Simulation(A, x0, time):
    '''
    線形状態方程式 dx/dt = Ax を解く
    by 有限差分法による数値シミュレーション
    '''
    def eom(x, t):
        return A.dot(x)
        
    motion = odeint(
        eom,   #運動方程式を表すユーザ関数
        x0,    #初期条件
        time   #時間軸を表す数列
    )
  
    return motion

def Solution_expA(A, x0, time):
    '''
    線形状態方程式 dx/dt = Ax の解を計算する
    by 行列指数関数による表示 x(t)= exp(tA)x0
    '''
    motion = [] #空のリスト
    for t in time:
        motion.append( #各時刻の解をリストに追加
            la.expm(t*A).dot(x0)
        )
  
    return np.array(motion) #Numpy配列に変換して返す

def plot_sim_vs_expA(A, x0, tminmax=[0,30], tn=200):
    '''
    数値シミュレーション vs 行列指数関数による解のプロット
    '''
    x0 = np.array(x0) #初期値
    ts = np.linspace(*tminmax, tn) #時間軸
    
    xs_sim  = Simulation(A, x0, ts)
    xs_expA = Solution_expA(A, x0, ts)
    
    fig, ax = plt.subplots(2,1,figsize=(6,4))

    for i in range(2):
        ax[i].plot(ts, xs_sim[:,i], 'o', 
                label=r'Simulation')
        ax[i].plot(ts, xs_expA[:,i], '-', 
                label=r'$e^{tA}x_0$')
        ax[i].legend()
        ax[i].set_xlabel(r'$t$')
        ax[i].set_ylabel(r'$x_%d$'%(i+1))

### 演習 8.1 ( 線形振動系の解の表示 )

- 状態方程式: $\displaystyle
\dot{\boldsymbol{x}} = A\boldsymbol{x}
,\quad
\boldsymbol{x}(0)=\boldsymbol{x}_0
,\quad
A:=\begin{bmatrix}
0 & 1\\
-k/m & -c/m
\end{bmatrix}
$
- 行列指数関数による解の表示: $\boldsymbol{x}(t)=e^{tA}\boldsymbol{x}_0$

In [ ]:
def A_L1DOF(param):
    '''
    線形1自由度（linear 1-degree-of-freedom）
    の振動系を表す行列
    '''
    m, c, k = param  #パラメータの成分
    A = np.array([
        [0, 1],
        [-k/m, -c/m],
    ])
    
    return A

#### ■数値例

In [ ]:
param, x0 = [1, 0.2, 2], [1,0]

plot_sim_vs_expA(A_L1DOF(param), x0)

In [ ]:
param, x0 = [1, 2, 1], [1,2] #ちなみに固有値が重根の場合

plot_sim_vs_expA(A_L1DOF(param), x0)

#### ■比較結果

- シミュレーションと $e^{tA}\boldsymbol{x}_0$ の結果は，パラメータや初期値を変えても一致します！

## ◯固有値と固有ベクトル

### 演習 8.3 ( 振動と行列の固有値の数値計算 )

#### ■$s^2 + 3s + 2 = 0$ の根

In [ ]:
np.roots([1, 3, 2])

#### ■行列 $\begin{bmatrix}0&1\\-2&-3\end{bmatrix}$ の固有値

In [ ]:
B = np.array([
    [ 0,  1],
    [-2, -3]
])

固有値 $s_i$ と固有ベクトル $\boldsymbol{v}_i$ を求めます
- `ss`: $\boldsymbol{s}:=[s_1,\cdots,s_n]$ 固有値を並べた配列 
- `V`: $V:=[\boldsymbol{v}_1,\cdots,\boldsymbol{v}_n]$ 単位固有ベクトルを並べた行列

In [ ]:
ss, V = la.eig(B)

print(ss)

固有方程式と同じ固有値が得られました．

- <font color='red'>2次方程式の根と順序が違いますが，これは単に`roots`と`eig`の仕様（結果の並べ方）の違いです．</font>
- 行列の固有値には虚部 `0.j` = 0 が見えてますが，これも単なる仕様の問題です．

固有値とその単位固有ベクトルを並べて表示してみます．

In [ ]:
for i, s in enumerate(ss):
    v = V[:,i] #固有ベクトル＝計算結果の列ベクトル
    print( 's =', s, ': v =', v)

例題8.3の手計算の結果と比較すると，同じ固有値と固有ベクトルが得られています．

## ◯複素共役による実数化

### 演習 8.4 ( 初期値の展開の数値計算 )

In [ ]:
A = np.array([ #固有値が複素数になるような行列
    [0, 1],
    [-1, -1]
])

#### ■固有値と単位固有ベクトル

In [ ]:
ss, V = la.eig(A)

#vv = np.array([V[:,i] for i in range(len(ss))]) #列ベクトルが固有ベクトル
vv = V.transpose() #上記よりシンプルな等価処理

for s, v in zip(ss, vv):
    print('s =', s, ': v =', v)

- 固有値・固有ベクトルが，確かに，共役複素数のペアで得られている

#### ■初期値の展開係数

In [ ]:
x0 = np.array([5, 6]) #適当な初期値

In [ ]:
etas = la.inv(V).dot(x0) #初期値の展開係数
print(etas)

- 展開係数も，共役複素数のペアになっている

#### ■初期値の復元（実数化）

In [ ]:
dim = len(ss) #次元

x0_rec = np.zeros(dim) #ゼロベクトル
for eta, v in zip(etas, vv): #展開係数*固有ベクトル の線形結合
    x0_rec = x0_rec + eta*v  

print(x0_rec)

- 虚部の計算機誤差$\approx -8\times 10^{-16}$ を除けば，元の実数ベクトル `[5,6]` が復元されている．

#### ■微小な計算機誤差が見づらいので，それを除去する処理

In [ ]:
def chop(array):
    '''
    微小な計算機誤差を除去する
    '''
    tol = 1e-10 #許容誤差（微小な数）

    #実部
    re = np.real(array)
    re[np.abs(re)<tol] = 0 #tol以下の項を0に

    #虚部
    if np.iscomplexobj(array):
        im = np.imag(array)
        im[np.abs(im)<tol] = 0 #tol以下の項を0に
    else:
        im = np.zeros_like(re)
    
    return re + 1j*im

chop(x0_rec)

## ◯固有値によるダイナミクスの分類

In [ ]:
def plot_multidim(ts, xs):
    '''
    多次元の解をプロットする
    '''
    tn, dim = np.shape(xs)

    fig, ax = plt.subplots(1,1,figsize=(6,2))

    ax.plot(ts, xs, '-')
    ax.set_xlabel(r'$t$')
    ax.set_ylabel(r'$x_i$')

    labels = [r'$x_%d$'%(i+1) for i in range(dim)]
    ax.legend(labels=labels, loc='lower right')
    ax.grid()

### 演習 8.5 ( モード展開の数値計算 )

In [ ]:
A = np.array([ #お試し用の行列
    [  0,   1,  0,  0],
    [0.1, 0.1,  0,  0],
    [  0,   0,  0,  1],
    [  0,   0, -1, -1],
])

#### 固有値と単位固有ベクトル

In [ ]:
ss, V = la.eig(A)
vv = V.transpose()

for s, v in zip(chop(ss), chop(vv)):
    print('s =', s)
    print(': v =', v)
    print('----------')

- (負の実根, 正の実根，複素数，その共役）が得られました．
- 共役な複素根に対しては，共役な複素固有ベクトルが得られています．

#### 初期値の展開係数

In [ ]:
x0 = np.array([1, -1, 2, 4]) #適当な初期値

In [ ]:
etas = la.inv(V).dot(x0) #初期値の展開係数

for eta in chop(etas):
    print(eta)

- 共役な固有ベクトルに掛かる展開係数は，やはり共役になっています．

#### 時間軸

In [ ]:
ts = np.linspace(0,10,200) #時間軸

#### ■「（a）実根 < 0」の成分

In [ ]:
ss[0]

In [ ]:
xa_minus = np.array([
    etas[0]*np.exp(ss[0]*t)*vv[0]
    for t in ts
])

xa_minus = chop(xa_minus) #計算機誤差の除去

虚部の大きさを確認

In [ ]:
print('size of imaginary = ', la.norm(np.imag(xa_minus)))

虚部は0なので，実部だけプロット

In [ ]:
plot_multidim(ts, np.real(xa_minus))

- 負の実根に対応する，非振動減衰が見て取れます．

#### ■「（a）実根 > 0」の成分

In [ ]:
ss[1]

In [ ]:
xa_plus = np.array([
    etas[1]*np.exp(ss[1]*t)*vv[1]
    for t in ts
])

xa_plus = chop(xa_plus) #計算機誤差の除去

虚部の大きさを確認しながらプロット

In [ ]:
print('size of imaginary = ', la.norm(np.imag(xa_plus)))
plot_multidim(ts, np.real(xa_plus))

- 正の実根に対応する，非振動発散が見られます．

#### ■「（c）共役な複素根」の成分

In [ ]:
ss[2], ss[3]

In [ ]:
xc = np.array([
    etas[2]*np.exp(ss[2]*t)*vv[2] + etas[3]*np.exp(ss[3]*t)*vv[3]
    for t in ts
])

xc = chop(xc) #計算機誤差の除去

虚部の大きさを確認しながらプロット

In [ ]:
print('size of imaginary = ', la.norm(np.imag(xc)))
plot_multidim(ts, np.real(xc))

- 実部が負の共役な複素根に対応する減衰振動が見て取れます．

#### ■楕円軌道の公式の検証

In [ ]:
def EE(omt, v, eta):
    '''
    楕円軌道の公式
    '''
    etaR,  etaI  = np.real(eta), np.imag(eta)
    vR,    vI    = np.real(v),   np.imag(v)
    Ec =  2*(etaR*vR - etaI*vI)
    Es = -2*(etaI*vR + etaR*vI)
    
    return np.cos(omt)*Ec + np.sin(omt)*Es 

In [ ]:
s, v, eta = ss[2], vv[2], etas[2]
gamma, omega = np.real(s), np.imag(s)

xc_E = np.array([
    np.exp(gamma*t)*EE(omega*t, v, eta)
    for t in ts
])

diff_xc = chop(xc - xc_E) #共役複素数の成分と公式との差
print('difference = ', la.norm(diff_xc))

- 公式は合ってます．

#### ■（ｄ）異なるダイナミクスの総和

In [ ]:
ss[0], ss[1], ss[2], ss[3]

In [ ]:
xsum = xa_minus + xa_plus + xc #全ての総和
# xsum = np.array([ #改めて総和し直す書き方　※上記と等価
#     sum(etas[i]*np.exp(ss[i]*t)*uu[i] for i in range(len(ss)))
#     for t in ts
# ])

xsum = chop(xsum) #計算機誤差の除去

虚部の大きさを確認しながらプロット

In [ ]:
print('size of imaginary = ', la.norm(np.imag(xsum)))
plot_multidim(ts, np.real(xsum))

- $\boldsymbol{x}(t)=$ （全成分の総和）のプロットです．
- 「（a）実根 > 0」成分の発散が勝って，$\boldsymbol{x}(t)$は全体としては発散します．